In [1]:
import numpy as np
import pandas as pd
import plotly as px
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from prophet import Prophet
from functools import reduce

%matplotlib inline


pd.set_option('display.max_columns', 100)

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import plotly.express as px

In [2]:
FDI = pd.read_csv('FDI_melt.csv')
GDP = pd.read_csv('GDP_melt.csv')
Inflation = pd.read_csv('inflation_melt.csv')
Trade = pd.read_csv('trade_melt.csv')
Unemployment = pd.read_csv('unemployment_melt.csv')
WDI = pd.read_csv('WDI_melt.csv')

In [3]:
name= ['FDI', 'GDP', 'Inflation', 'Trade', 'Unemployment', 'WDI']
for name in name:
    globals()[name].rename(columns={'Value': f'Value_{name}'}, inplace=True)

In [4]:
# List semua DataFrame
dataframes = [FDI, GDP, Inflation, Trade, Unemployment, WDI]

# Gabungkan semua DataFrame berdasarkan kolom 'country' dan 'year'
merged_df = reduce(lambda left, right: pd.merge(left, right, on=['Country Name', 'Year'], how='outer'), dataframes)
merged_df.head()

,Country Name,Year,Value_FDI,Value_GDP,Value_Inflation,Value_Trade,Value_Unemployment,Value_WDI
0,Aruba,2000,-1.279330e+08,7.622921,4.044021,145.072879,NaN,111.790970
1,Africa Eastern and Southern,2000,4.315009e+09,3.219803,8.601485,47.120277,7.409352,78.739380
2,Afghanistan,2000,1.700000e+05,NaN,NaN,NaN,7.955000,22.162991
3,Africa Western and Central,2000,2.560095e+09,3.859760,2.530775,NaN,4.627660,82.695992
4,Angola,2000,8.786200e+08,3.054624,324.996872,NaN,16.502000,NaN


In [5]:
columns = ['Value_FDI',	'Value_GDP','Value_Inflation',	'Value_Trade',	'Value_Unemployment','Value_WDI']

for cols in columns:
    # Mean Value groupby timeperiod
    mean_values_per_year = merged_df.groupby('Year')[cols].mean().reset_index()

    fig = px.line(mean_values_per_year, 
                x='Year', 
                y=cols,
                markers=True,
                title=f'Rata-rata Value per year {cols}')

    # Menambahkan label sumbu
    fig.update_layout(
        xaxis_title='Time Period',
        yaxis_title='Average Value',
        title_font_size=20,
        xaxis_title_font_size=18,
        yaxis_title_font_size=18
    )

    # Menampilkan chart
    fig.show()


In [6]:
merged_df['Country Name'].nunique()

265

In [7]:
merged_df['Country Name'].unique()

array(['Aruba', 'Africa Eastern and Southern', 'Afghanistan',
       'Africa Western and Central', 'Angola', 'Albania', 'Andorra',
       'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan',
       'Botswana', 'Central African Republic', 'Canada',
       'Central Europe and the Baltics', 'Switzerland', 'Channel Islands',
       'Chile', 'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.',
       'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica',
       'Caribbean small states', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria',
 

In [8]:
merged_df.fillna(0, inplace=True)

In [9]:
fig_gdp = px.line(
    merged_df, 
    x='Year', 
    y='Value_GDP', 
    color='Country Name', 
    title='Distribusi GDP dari Tahun ke Tahun untuk Setiap Negara',
    labels={'Value_GDP': 'Nilai GDP'},
    markers=True
)
fig_gdp.show()

In [10]:
merged_df.to_csv('merged_df.csv', index=False)

In [11]:
def forecast_by_country_prophet(df, column, start_year=2024, end_year=2030):
    forecast_results = []
    historical_years = df['Year'].unique()
    historical_data = []

    country_data = df[df['Country Name'] == 'World'][['Year', column]].rename(columns={'Year': 'ds', column: 'y'})

    # Check for NaN values and drop them
    country_data = country_data.dropna()

    if len(country_data) < 2:
        raise ValueError(f"Not enough data to fit the model for 'World'. Found {len(country_data)} non-NaN rows.")

    historical_data = country_data['y'].values

    model = Prophet()
    model.fit(country_data)

    future_years = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='Y')
    future = pd.DataFrame({'ds': future_years})

    forecast = model.predict(future)
    forecast_results.append(forecast['yhat'].values)

    average_forecast = pd.DataFrame(forecast_results).mean()
    return future_years.year, average_forecast, historical_years, historical_data

def plot_average_forecast_plotly_with_history(future_years, average_forecast, historical_years, historical_data, column_name):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=historical_years, y=historical_data,
                             mode='lines+markers',
                             name='Historical Data',
                             line=dict(width=2, dash='solid')))

    fig.add_trace(go.Scatter(x=future_years, y=average_forecast,
                             mode='lines+markers',
                             name='Forecast',
                             line=dict(color='orange', width=2, dash='dot')))

    fig.update_layout(title=f'Average Forecast of {column_name} with Historical Data',
                      xaxis_title='Year',
                      yaxis_title=f'Average {column_name}',
                      xaxis=dict(tickmode='linear'),
                      template='plotly_white')
    
    fig.show()

file_path = 'merged_df.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Forecasting for GDP and calculating the average using Prophet
try:
    future_years, avg_gdp_forecast, historical_years, historical_data = forecast_by_country_prophet(df, 'Value_GDP')
    
    # Plot the historical and forecasted GDP data
    plot_average_forecast_plotly_with_history(future_years, avg_gdp_forecast, historical_years, historical_data, 'GDP')

    avg_gdp_forecast_df = pd.DataFrame({'Year': future_years, 'Average GDP Forecast': avg_gdp_forecast})
    print(avg_gdp_forecast_df)
except ValueError as e:
    print(e)


19:02:41 - cmdstanpy - INFO - Chain [1] start processing
19:02:41 - cmdstanpy - INFO - Chain [1] done processing


   Year  Average GDP Forecast
0  2024              1.843442
1  2025              1.519143
2  2026              1.171017
3  2027              0.799488
4  2028              1.690038
5  2029              1.365739
6  2030              1.017613


In [12]:
avg_gdp_forecast_df.rename(columns={'Average GDP Forecast': 'Value_GDP'}, inplace=True)

In [13]:
# create a new dataframe that combined forecasted GDP datas into main dataframe (merged_df)
merged_df_copy = merged_df[merged_df['Country Name'] == 'World']._append(avg_gdp_forecast_df, ignore_index=True)
merged_df_copy

,Country Name,Year,Value_FDI,Value_GDP,Value_Inflation,Value_Trade,Value_Unemployment,Value_WDI
0,World,2000,1.569240e+12,4.531095,3.433516,50.497096,6.131118,98.772423
1,World,2001,8.956330e+11,2.025403,3.836573,49.388370,6.223468,99.151611
2,World,2002,7.517700e+11,2.302837,2.907999,49.444717,6.468308,99.543800
3,World,2003,7.366030e+11,3.104276,3.025045,51.146551,6.571283,101.092293
4,World,2004,1.008890e+12,4.469259,3.517999,54.742993,6.449677,101.901756
5,World,2005,1.553380e+12,4.005866,4.107251,56.806928,6.366465,102.250458
6,World,2006,2.189100e+12,4.442793,4.267175,59.013033,6.106586,102.250382
7,World,2007,3.112540e+12,4.373277,4.810237,59.423494,5.899677,103.092087
8,World,2008,2.441600e+12,2.062496,8.949953,61.078961,5.936694,103.294220
9,World,2009,1.517730e+12,-1.355782,2.860449,52.567452,6.514687,103.119682


In [14]:
merged_df_copy.to_csv('merge_and_forecast.csv', index=False)